# Imports y config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import nibabel as nib
import numpy as np
import torch

In [3]:
os.chdir('C:\\Users\\Usuario\\TFG\\digipanca\\')

# Check slices

In [6]:
DATA_DIR = 'data/raw'
PATIENT_IDS = [p for p in os.listdir(DATA_DIR)]

## Load patients NIfTIs

In [18]:
SLICES = {}

for pid in PATIENT_IDS:
    image_path = os.path.join(DATA_DIR, pid, "SEQ", f"CTport-{pid}.nii")
    image = nib.load(image_path)
    SLICES[pid] = image.shape[2]

In [19]:
print(SLICES)

{'rtum1': 91, 'rtum10': 56, 'rtum11': 104, 'rtum12': 68, 'rtum13': 67, 'rtum14': 125, 'rtum15': 120, 'rtum16': 114, 'rtum17': 108, 'rtum18': 104, 'rtum19': 91, 'rtum2': 90, 'rtum20': 44, 'rtum21': 116, 'rtum22': 117, 'rtum23': 105, 'rtum24': 101, 'rtum25': 105, 'rtum26': 116, 'rtum27': 54, 'rtum28': 118, 'rtum29': 89, 'rtum3': 109, 'rtum30': 120, 'rtum31': 110, 'rtum32': 120, 'rtum33': 106, 'rtum34': 41, 'rtum35': 55, 'rtum36': 117, 'rtum37': 118, 'rtum38': 103, 'rtum39': 56, 'rtum4': 60, 'rtum40': 46, 'rtum41': 135, 'rtum42': 98, 'rtum43': 118, 'rtum44': 100, 'rtum45': 102, 'rtum46': 142, 'rtum47': 134, 'rtum48': 92, 'rtum49': 114, 'rtum5': 77, 'rtum50': 129, 'rtum51': 112, 'rtum52': 116, 'rtum53': 130, 'rtum54': 132, 'rtum55': 132, 'rtum56': 98, 'rtum57': 44, 'rtum58': 100, 'rtum59': 131, 'rtum6': 112, 'rtum60': 92, 'rtum61': 56, 'rtum62': 116, 'rtum63': 102, 'rtum64': 112, 'rtum65': 53, 'rtum66': 129, 'rtum67': 116, 'rtum68': 101, 'rtum69': 115, 'rtum7': 49, 'rtum70': 108, 'rtum71':

In [20]:
max_slices = np.max(list(SLICES.values()))
min_slices = np.min(list(SLICES.values()))
print(f"Max slices: {max_slices}")
print(f"Min slices: {min_slices}")

Max slices: 152
Min slices: 41


# Subvolumes

In [22]:
def extract_subvolumes(volume, target_slices=64, stride=32):
    subvolumes = []
    total_slices = volume.shape[0]
    
    for start in range(0, total_slices - target_slices + 1, stride):
        subvolumes.append(volume[start:start + target_slices, :, :])
    
    # Si el volumen es muy pequeño, hacemos padding con ceros
    if total_slices < target_slices:
        pad_size = target_slices - total_slices
        padded_volume = np.pad(volume, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        subvolumes.append(padded_volume)
    
    return np.array(subvolumes)

# Extraer subvolúmenes
print(image.shape)
image_array = image.get_fdata()
image_array = np.transpose(image_array, (2, 0, 1))
subvols = extract_subvolumes(image_array, target_slices=64, stride=32)
print(subvols.shape)  # (N, 64, H, W)

(512, 512, 68)
(1, 64, 512, 512)


In [23]:
for pid in PATIENT_IDS:
    image_path = os.path.join(DATA_DIR, pid, "SEQ", f"CTport-{pid}.nii")
    image = nib.load(image_path)
    image_array = image.get_fdata()
    image_array = np.transpose(image_array, (2, 0, 1))
    subvols = extract_subvolumes(image_array, target_slices=64, stride=32)
    print(f"{pid}: {image.shape} > {subvols.shape}")

rtum1: (512, 512, 91) > (1, 64, 512, 512)
rtum10: (512, 512, 56) > (1, 64, 512, 512)
rtum11: (512, 512, 104) > (2, 64, 512, 512)
rtum12: (512, 512, 68) > (1, 64, 512, 512)
rtum13: (512, 512, 67) > (1, 64, 512, 512)
rtum14: (512, 512, 125) > (2, 64, 512, 512)
rtum15: (512, 512, 120) > (2, 64, 512, 512)
rtum16: (512, 512, 114) > (2, 64, 512, 512)
rtum17: (512, 512, 108) > (2, 64, 512, 512)
rtum18: (512, 512, 104) > (2, 64, 512, 512)
rtum19: (512, 512, 91) > (1, 64, 512, 512)
rtum2: (512, 512, 90) > (1, 64, 512, 512)
rtum20: (512, 512, 44) > (1, 64, 512, 512)
rtum21: (512, 512, 116) > (2, 64, 512, 512)
rtum22: (512, 512, 117) > (2, 64, 512, 512)
rtum23: (512, 512, 105) > (2, 64, 512, 512)
rtum24: (512, 512, 101) > (2, 64, 512, 512)
rtum25: (512, 512, 105) > (2, 64, 512, 512)
rtum26: (512, 512, 116) > (2, 64, 512, 512)
rtum27: (512, 512, 54) > (1, 64, 512, 512)
rtum28: (512, 512, 118) > (2, 64, 512, 512)
rtum29: (512, 512, 89) > (1, 64, 512, 512)
rtum3: (512, 512, 109) > (2, 64, 512, 512)


# image.slices != mask.slices

In [56]:
for pid in PATIENT_IDS:
    sample_dir = os.path.join(DATA_DIR, pid)
    image_path = os.path.join(sample_dir, "SEQ", f"CTport-{pid}.nii")
    mask_paths = {
        "pancreas": os.path.join(sample_dir, "SEG", f"Pancreas-{pid}.nii"),
        "tumor": os.path.join(sample_dir, "SEG", f"Tumor-{pid}.nii"),
        "arteries": os.path.join(sample_dir, "SEG", f"Arterias-{pid}.nii"),
        "veins": os.path.join(sample_dir, "SEG", f"Venas-{pid}.nii"),
    }
    image = nib.load(image_path)
    vol_slices = image.shape[2]
    seg_slices = []
    for _, path in mask_paths.items():
        mask = nib.load(path)
        seg_slices.append(mask.shape[2])
    if vol_slices != seg_slices[0] or vol_slices != seg_slices[1] or vol_slices != seg_slices[2] or vol_slices != seg_slices[0]:
        print(f"ERROR in patient {pid}")
        print(f"\tVol: {vol_slices} | Masks : {seg_slices[0]} - {seg_slices[1]} - {seg_slices[2]} - {seg_slices[3]}")

ERROR in patient rtum13
	Vol: 67 | Masks : 268 - 268 - 268 - 268
ERROR in patient rtum32
	Vol: 120 | Masks : 120 - 480 - 120 - 120
ERROR in patient rtum85
	Vol: 152 | Masks : 152 - 608 - 152 - 152


In [29]:
rtum13 = os.path.join(DATA_DIR, 'rtum13')

In [54]:
import os
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to

def process_and_save_niftis(patient_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    pid = os.path.basename(patient_dir)
    os.makedirs(os.path.join(output_dir, pid), exist_ok=True)
    os.makedirs(os.path.join(output_dir, pid, "SEG"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, pid, "SEQ"), exist_ok=True)

    # Cargar la imagen principal
    image_path = os.path.join("SEQ", f"CTport-{pid}.nii")
    img_nii = nib.load(os.path.join(patient_dir, image_path))
    
    # Guardamos la imagen sin modificaciones
    nib.save(img_nii, os.path.join(output_dir, pid, image_path))

    # Diccionario con las rutas de las máscaras
    mask_paths = {
        "pancreas": os.path.join("SEG", f"Pancreas-{pid}.nii"),
        "tumor": os.path.join("SEG", f"Tumor-{pid}.nii"),
        "arteries": os.path.join("SEG", f"Arterias-{pid}.nii"),
        "veins": os.path.join("SEG", f"Venas-{pid}.nii"),
    }

    for label, path in mask_paths.items():
        mask_nii = nib.load(os.path.join(patient_dir, path))
        
        # Reescalar máscara al espacio de la imagen
        mask_resampled = resample_from_to(mask_nii, img_nii, order=0)  # order=0 mantiene etiquetas discretas
        
        print(f"🔹 {label} | Original: {mask_nii.shape}, Reescalado: {mask_resampled.shape}")

        # Guardamos la nueva máscara corregida
        nib.save(mask_resampled, os.path.join(output_dir, pid, path))

    print(f"✅ Archivos corregidos guardados en {os.path.join(output_dir, pid)}")


In [55]:
process_and_save_niftis(rtum13, 'data/processed/')

🔹 pancreas | Original: (512, 512, 268), Reescalado: (512, 512, 67)
🔹 tumor | Original: (512, 512, 268), Reescalado: (512, 512, 67)
🔹 arteries | Original: (512, 512, 268), Reescalado: (512, 512, 67)
🔹 veins | Original: (512, 512, 268), Reescalado: (512, 512, 67)
✅ Archivos corregidos guardados en data/processed/rtum13
